# Building a RAG pipeline

In [10]:
import os

# Set the LANGCHAIN_TRACING_V2 environment variable to enable tracing for LangChain version 2
os.environ["LANGCHAIN_TRACING_V2"] = "true"

# Set the LANGCHAIN_API_KEY environment variable with your LangChain API key
os.environ["LANGCHAIN_API_KEY"] = "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"

# Set the OPENAI_API_KEY environment variable with your OpenAI API key
#os.environ["OPENAI_API_KEY"] = "a164a2381286481bac2afbbf622114fb" #"sk-proj-tKZzJhFV0n3RBNvcFiiYT3BlbkFJFXX6o83ZmMZ6gycFqesG"

os.environ["AZURE_OPENAI_API_KEY"] = "01a86ebdb2fc4d2ea1660672e161a116"
os.environ["AZURE_OPENAI_EMBEDDING_MODEL_NAME"] = 'nithin-text-embedding-ada-002'
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://mandarnithinpraveen.openai.azure.com/"

# Set the PINECONE_API_KEY environment variable with your Pinecone API key
os.environ["PINECONE_API_KEY"] = "9602b65f-cc2a-41c6-8a40-f56b499dc060"

# Set the LANGCHAIN_PROJECT environment variable to specify the project name for LangChain
os.environ['LANGCHAIN_PROJECT'] = "earnings-call"

# Set the LANGSMITH_USER_HANDLE environment variable with your Langsmith user handle
os.environ['LANGSMITH_USER_HANDLE'] = "earnings"
#01a86ebdb2fc4d2ea1660672e161a116
#a164a2381286481bac2afbbf622114fb

## Importing the required packages

In [11]:
# Import the json module for handling JSON data
import json

# Import StrOutputParser from langchain_core.output_parsers for parsing string outputs
from langchain_core.output_parsers import StrOutputParser

# Import RunnablePassthrough from langchain_core.runnables for running tasks in a passthrough manner
from langchain_core.runnables import RunnablePassthrough

# Import RunnableParallel from langchain_core.runnables for running tasks in parallel
from langchain_core.runnables import RunnableParallel

# Import ChatPromptTemplate from langchain_core.prompts for creating chat prompt templates
from langchain_core.prompts import ChatPromptTemplate

# Import OpenAIEmbeddings from langchain_openai for working with OpenAI embeddings

# Import ChatOpenAI from langchain_openai for interacting with the OpenAI chat model

from langchain_openai import OpenAIEmbeddings, AzureOpenAIEmbeddings
 
# Import ChatOpenAI from langchain_openai for interacting with the OpenAI chat model
from langchain_openai import ChatOpenAI, AzureChatOpenAI

# Import PineconeVectorStore from langchain_pinecone for managing vector storage in Pinecone
from langchain_pinecone import PineconeVectorStore

In [12]:
embeddings = AzureOpenAIEmbeddings(azure_endpoint="https://mandarnithinpraveen.openai.azure.com/",
                              openai_api_key="01a86ebdb2fc4d2ea1660672e161a116", 
                              model='nithin-text-embedding-ada-002'
                              )
llm = AzureChatOpenAI(model="nithin-gpt35-turbo", temperature=0,api_version="2024-02-01")

### Defining globle variables

In [13]:
# Define the index name for storing or retrieving data, in this case, 'earning-calls'
INDEX_NAME = 'my-earnings-call'

# Set the number of top results to return, here set to 6
TOP_K = 6

# Specify the quarter for which the data is relevant, in this example, Q1 (Quarter 1)
QUARTER = "Q1"

# Define the filename of the document being processed, in this case, "Adani Enterprises Ltd.pdf"
#FILENAME = "Adani Enterprises Ltd.pdf"
FILENAME = "Adani Enterprises Ltd.pdf"

# Specify the fiscal year for the data, in this example, FY24 (Fiscal Year 2024)
# YEAR = "FY24"
YEAR = "FY24"

# Initialize OpenAIEmbeddings with the specified model for generating text embeddings
#embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# Initialize ChatOpenAI with the specified model and temperature for generating chat responses
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Initialize ChatOpenAI with the specified model and temperature for generating chat responses
llm = AzureChatOpenAI(model="nithin-gpt35-turbo", temperature=0,api_version="2024-02-01")

## Loading Vectorstore

In [14]:
# Create a PineconeVectorStore instance with the specified index name and embedding model
#index = <YOUR CODE HERE>
docsearch = PineconeVectorStore(index_name=INDEX_NAME, embedding=embeddings)

# Convert the PineconeVectorStore instance into a retriever object for searching
retriver =  docsearch.as_retriever(search_kwargs={"filter": {"quarter": QUARTER, "filename": FILENAME}, "k": TOP_K} )

In [15]:
retriver.invoke("what is the capex?")

[Document(page_content=' Adani Enterprises Limited  \nAugust 03 , 2023 \n \n Page 6 of 11 \nairport side , we will have capex this year just about US $ 1.1 billion , all of this just to clarify we \nare assuming  Rs.80 to a dollar rate , adjust for that , so about US $1.1 billion this year would be \nthe CAPEX on airport. It will broadly remain in that range for the next  year then there will be \ndecline , once we complete the first phase  of our development plan in airports . \nModerator : Thank you. The next question is from the line of Nikhil Abhyankar from ICICI Securities. Please \ngo ahead.  \nNikhil Abhyankar : Congrats on a good set of numbers . What is the guidance for commercial mining this year?  And \nthe reason for asking the question is  our production has fallen like 10% Y-o-Y, so what is the \nexact reason for the same?  \nVinay Prakash : As for your first question  about the commercial mines, we are hopeful of starting the open  cut \nin Dhirauli mine which is the com

## Creating a prompt template

In [16]:
# This template is designed for a chatbot that acts as an expert Q&A system
chat_template = ChatPromptTemplate.from_messages(
    [
        # Define the system message that outlines the chatbot's role and rules
        ("system", "You are a helpful financial assistant and your task is to extract financial inormation for the earnings call reports. Extract all the necessary information required like CAPEX, nature of business , quarterly earning, profit and Loss and so on.."),
        
        # Define the human message that provides context information and a query
        ("human", "Context information is below.\n---------------------\n{context}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {query}\nAnswer: "),
    ]
)


In [17]:
print(chat_template.format(context="THIS IS A SAMPLE CONTEXT TO SEE HOW THE PROMPT LOOKS", query="THIS IS A SAMPLE QUERY?"))

System: You are a helpful financial assistant and your task is to extract financial inormation for the earnings call reports. Extract all the necessary information required like CAPEX, nature of business , quarterly earning, profit and Loss and so on..
Human: Context information is below.
---------------------
THIS IS A SAMPLE CONTEXT TO SEE HOW THE PROMPT LOOKS
---------------------
Given the context information and not prior knowledge, answer the query.
Query: THIS IS A SAMPLE QUERY?
Answer: 


## Creating the RAG chain

In [18]:
# Define a function to format documents by joining their page content with newlines
def format_docs(docs):
    # Join the page content of each document with two newlines between them
    return "\n\n".join(doc.page_content for doc in docs)

# Create a runnable chain for generating responses from formatted documents
# This chain starts with formatting the documents, then uses the chat template,
# processes the response through the language model, and finally parses the output
rag_chain_from_docs = (
    RunnablePassthrough.assign(context = (lambda x : format_docs(x["context"])))
    | chat_template
    | llm
    | StrOutputParser()
)

# Create a runnable chain for retrieving context and generating answers in parallel
# This chain retrieves context using the retriever and formats the query,
# then generates answers using the previously defined chain for formatted documents
rag_chain_with_source = RunnableParallel(
    {"context" : retriver, "query" : RunnablePassthrough()}
).assign(answer = rag_chain_from_docs)

In [19]:
response = rag_chain_with_source.invoke("What was the income?")
print(response['answer'])

The consolidated total income for Q1 FY24 was Rs. 25,810 crores.


In [1]:
response

NameError: name 'response' is not defined

## Prompt Versioning

Loading a Specific Version of a Prompt:

1. **Version Tracking in Repositories:**
   - Each push to a prompt repository saves a new version, identified by a unique commit hash.

2. **Loading the Latest Version:**
   - By default, accessing the repo will load the most recent version of a given prompt.

3. **Loading a Specific Version:**
   - To load a specific version, include its commit hash with the prompt name.
   - Example: For loading the "earnings-call-rag" with version `6214c98a`, append this hash to the prompt name in your loading command.

In [ ]:
# !pip install langchainhub

In [15]:
from langchain import hub
prompt = hub.pull("bhaskarjit/earnings-call-rag")

In [16]:
print(prompt.format(context="THIS IS A SAMPLE CONTEXT TO SEE HOW THE PROMPT LOOKS", query="THIS IS A SAMPLE QUERY?"))

System: You are an expert Q&A system that is trusted around the world.
Always answer the query using the provided context information, and not prior knowledge.
Some rules to follow:
1. Never directly reference the given context in your answer.
2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines.
Human: Context information is below.
---------------------
THIS IS A SAMPLE CONTEXT TO SEE HOW THE PROMPT LOOKS
---------------------
Given the context information and not prior knowledge, answer the query.
Use Bullet poits whenever possible in the answer.
Query: THIS IS A SAMPLE QUERY?
Answer: 


In [17]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt # placing the newly loaded prompt here
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriver, "query": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [18]:
response = rag_chain_with_source.invoke("What was the income?")
print(response['answer'])

- The consolidated total income for the quarter was Rs. 25,810 crores.


### How to Share Prompts on LangChain Hub:

1. **Getting Started:**
   - Getting prompts from LangChain Hub is easy, and so is sharing your own prompts.

   - This lets you easily share and manage your own prompts.

2. **Making a Prompt:**

   - First, create a prompt that fits what you need.

   - Make sure it follows the rules of LangChain Hub.

3. **Sharing Process:**

   - The sharing has two important parts:
     - **Account Handle:** Your special name in LangChain Hub, like `me-langchain-user`.
     - **Prompt Name:** A clear name for your prompt, showing what it does.

4. **How to Share with Code:**
   - This is a simple way to share:
     ```python
     from langchain import hub

     # Define your prompt
     my_prompt = "..."  # Your prompt content goes here

     # Share it on LangChain Hub
     hub.push(f"{account_handle}/{prompt_name}", my_prompt)
     ```
     - Replace `account_handle` with your username and `my_prompt` with your prompt's name.
     
     - Make sure `my_prompt` follows LangChain PromptTemplate.

5. **Using Your Shared Prompt:**
   - Once it's shared, you can use it in different apps through LangChain Hub.
   - This makes it easy to share with others, work together, and manage your prompts.

In [19]:
len(prompt.messages), prompt.messages

(2,
 [SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="You are an expert Q&A system that is trusted around the world.\nAlways answer the query using the provided context information, and not prior knowledge.\nSome rules to follow:\n1. Never directly reference the given context in your answer.\n2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along\xa0those\xa0lines.")),
  HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'query'], template='Context information is below.\n---------------------\n{context}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nUse Bullet poits whenever possible in the answer.\nQuery: {query}\nAnswer: '))])

In [20]:
prompt.messages[1]

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'query'], template='Context information is below.\n---------------------\n{context}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nUse Bullet poits whenever possible in the answer.\nQuery: {query}\nAnswer: '))

In [21]:
prompt.messages[1].prompt

PromptTemplate(input_variables=['context', 'query'], template='Context information is below.\n---------------------\n{context}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nUse Bullet poits whenever possible in the answer.\nQuery: {query}\nAnswer: ')

In [22]:
print(prompt.messages[1].prompt.template)

Context information is below.
---------------------
{context}
---------------------
Given the context information and not prior knowledge, answer the query.
Use Bullet poits whenever possible in the answer.
Query: {query}
Answer: 


In [23]:
prompt.messages[1].prompt.template = 'Context information is below.\n---------------------\n{context}\n---------------------\nGiven the context information and not prior knowledge, answer the query.Be precise.Do stick to facts as mentioned in the context.\nUse Bullet points whenever possible in the answer.\nQuery: {query}\nAnswer: '

In [24]:
print(prompt.messages[1].prompt.template)

Context information is below.
---------------------
{context}
---------------------
Given the context information and not prior knowledge, answer the query.Be precise.Do stick to facts as mentioned in the context.
Use Bullet points whenever possible in the answer.
Query: {query}
Answer: 


Added the following line in the prompt: `Use Bullet poits whenever possible in the answer.`

In [25]:
handle = "earnings"
prompt_url = hub.push(f"{handle}/earnings-call-rag", prompt)

In [26]:
prompt_url

'https://smith.langchain.com/hub/earnings/earnings-call-rag/6d0a5b75'